# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with $\verb+dict()+$ primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of $\verb+dict()+$, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function $\verb+collections.defaultdict+$ can be useful.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, and $\texttt{T}$. Given a *DNA sequence* e.g. $\texttt{ACGATGAGGCTCAT}$, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil ($\texttt{U}$). 


1. Write a function $\verb+dna_to_rna+$ to convert a DNA sequence into an RNA sequence. For the sake of exercise, use $\verb+dict()+$ to store the symbol to symbol encoding rules. Create a program to test your function.

In [93]:
from IPython.core import page
page.page=print # turn off the pager, so that %pycat prints inline

In [94]:
# exercise 1 DO NOT MODIFY THIS LINE
%pycat part07-e01_dna_to_rna/src/dna_to_rna.py

#!/usr/bin/env python3

DNA_TO_RNA = {
    "C": "C",
    "G": "G",
    "A": "A",
    "T": "U"
}

def dna_to_rna(s):
    as_rna = s
    for dna_base, rna_base in DNA_TO_RNA.items():
        as_rna = as_rna.replace(dna_base, rna_base)
    return as_rna

if __name__ == '__main__':
    print(dna_to_rna("ATCCCGTAGGCTCAT"))



### Idea of solution
The iterates over the RNA to DNA mappings dict, replacing the DNA nucleotides of the input string with the RNA nucleotides one by one. 

### Discussion
For example, dna_to_rna("ATCCCGTAGGCTCAT") produces AUCCCGUAGGCUCAU.
The solution is as expected, essentially each T is replaced with an U since this is only difference between DNA and RNA.


## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function $\verb+get_dict+$ to read the table into a $\verb+dict()+$, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [95]:
# exercise 2 DO NOT MODIFY THIS LINE
%pycat part07-e02_rna_to_prot/src/rna_to_prot.py

#!/usr/bin/env python3
from bs4 import BeautifulSoup
import re

CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    parsed = BeautifulSoup(html, 'html.parser').pre.text
    codons = re.findall(CODON_REGEX, parsed)
    aminoacids = re.findall(AA_REGEX, parsed)

    return dict(zip(codons, aminoacids))

if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)



### Idea of solution

**NOTE:** I this exercise and coming ones, I renamed the html file to **Codon_usage_table.html**, so this must be done if to run the tests. This is because whitespace in file names used in code is usually a bad idea and should be avoided. 

The solution uses BeautifulSoup parser library to parse the document and  and extract the contents of pre tag which contains the data we want. The it uses regexes to extract the codon data. The codon is matched by the regex wheb there is 3 consecutive alphanumeric letters flanked by whitespace. For the aminoacid the requirement is the same except only one letter is allowed. 

After that the two lists are zipped into a dict. 


### Discussion
The function works as expected returning a dict which has the length 64 where the codons are the keys and aminoacids are the values.  

The result is:

`{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}`


3. Use the same conversion table as above, but now write function $\verb+get_dict+$ to read the table into a $\verb+dict()+$, such that for each amino acid the hash table stores the list of codons encoding it.    

In [96]:
# exercise 3 DO NOT MODIFY THIS LINE

%pycat part07-e03_prot_to_rna/src/prot_to_rna.py

#!/usr/bin/env python3
import re
from collections import defaultdict
from bs4 import BeautifulSoup

CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    parsed = BeautifulSoup(html, 'html.parser').pre.text
    codons = re.findall(CODON_REGEX, parsed)
    aminoacids = re.findall(AA_REGEX, parsed)

    aa_to_codon_dict = defaultdict(list)
    for index, aa in enumerate(aminoacids):
        aa_to_codon_dict[aa].append(codons[index])

    return dict(aa_to_codon_dict)

if __name__ == '__main__':
    aa_to_codons = get_dict()
    from pprint import PrettyPrinter
    PrettyPrinter().pprint(aa_to_codons)



### Idea of solution

The parsing part of the function works as in the previous exercise. 

The returned dictionary is formed in a for loop where the codon is appended to a list for the codon that encodes it. This loop code makes use of defaultdict which initializes every unset key's value to an empty array. 

# Discussion

The result is:

`
{'*': ['UAA', 'UGA', 'UAG'],
 'A': ['GCU', 'GCC', 'GCA', 'GCG'],
 'C': ['UGU', 'UGC'],
 'D': ['GAU', 'GAC'],
 'E': ['GAA', 'GAG'],
 'F': ['UUU', 'UUC'],
 'G': ['GGU', 'GGC', 'GGA', 'GGG'],
 'H': ['CAU', 'CAC'],
 'I': ['AUU', 'AUC', 'AUA'],
 'K': ['AAA', 'AAG'],
 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'],
 'M': ['AUG'],
 'N': ['AAU', 'AAC'],
 'P': ['CCU', 'CCC', 'CCA', 'CCG'],
 'Q': ['CAA', 'CAG'],
 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'],
 'T': ['ACU', 'ACC', 'ACA', 'ACG'],
 'V': ['GUU', 'GUC', 'GUA', 'GUG'],
 'W': ['UGG'],
 'Y': ['UAU', 'UAC']} 
 ...
 `

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function $\verb+dna_to_prot+$ in the stub solution to convert a DNA sequence into a protein sequence. 
You may copy-paste the result dictionaries from exercises 2 and 3.
Run your program with $\texttt{ATGATATCATCGACGATGTAG}$.   

In [97]:
# exercise 4 DO NOT MODIFY THIS LINE
%pycat part07-e04_dna_to_prot/src/dna_to_prot.py

#!/usr/bin/env python3
import re
RNA_TO_PROT = {'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}

DNA_TO_RNA = {
    "C": "C",
    "G": "G",
    "A": "A",
    "T": "U"
}

def dna_to_rna(s):
    as_rna = s
    for dna_base, rna_base in DNA_TO_RNA.items():
        as_rna = as_rna.re

### Idea of solution
The solution uses the dna_to_rna function which was implemented in exercise 1. Once RNA is produced, the rna_to_prot first splits the DNA string to character triplets with regex '...'. If the input string length is not divisible by three, the excess letters in the end are dropped. The triplets are then mapped to amino acids by using the conversion dict produced in the previous exercise.

### Discussion
The result for dna_to_prot("ATGATATCATCGACGATGTAG") is MISSTM*.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons $\texttt{AUU}$, $\texttt{AUC}$, and $\texttt{AUA}$ that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is $\texttt{AUU}$ $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a $\verb+dict()+$, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the $\verb+get_dict+$ function.

In [98]:
# exercise 5 DO NOT MODIFY THIS LINE
%pycat part07-e05_codon_to_prob/src/codon_to_prob.py

#!/usr/bin/env python3
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import pandas as pd
CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'
FREQ_REGEX = r'(\d+.\d)\s{1}\('

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_synonomous_freqs_sum(aa, df):
    synonymous_codons = df[df.aa == aa]
    return synonymous_codons.freqs_per_1000.sum()

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    codon_html = BeautifulSoup(html, 'html.parser').pre.text

    df = pd.DataFrame()
    df['codon'] = re.findall(CODON_REGEX,  codon_html)
    df['freqs_per_1000'] = list(map(float, re.findall(FREQ_REGEX, codon_html)))
    df['aa'] = re.findall(AA_REGEX, codon_html)
    df['syn_sum'] = df.aa.map(lambda aa: get_synonomous_freqs_sum(aa, df))
    df['freq_frac'] = round(df.freqs_per_1000 / df.syn_sum, 3)
    return dict(zip(df.codon, df.freq_frac)

### Idea of solution

Going from protein to DNA (i.e reverse translation) is more complicated than the reverse direction (DNA to protein), because several codons encode the same amino acid and we cannot inambiguously tell from the protein which codon produced it. 

The parsing of codon and aminoacids is as in previous exercises. The frequency data is parsed with a regex that requires a decimal number that is followed by whitespace. A dataframe is formed with columns codon, aminoacid and freqs_per_1000. Aminoacid column is mapped to a new column which contains the sum of all synonymous codons (i.e. codons that encode the same amino acid). After this getting the desired probability output is just a matter of dividing the frequency of the aminoacid with the summed up frequency of its synonymous amino acids. 

### Discussion
Part of the output is:

`{'AAA': 0.433,
 'AAC': 0.529,
 'AAG': 0.567,
 'AAU': 0.471,
 'ACA': 0.284,
 'ACC': 0.355,
 'ACG': 0.115,
 'ACU': 0.246,
 'AGA': 0.215,
 'AGC': 0.24,
 'AGG': 0.212,
 'AGU': 0.149,
 'AUA': 0.169,
 'AUC': 0.47,
 'AUG': 1.0,
 'AUU': 0.361,
 'CAA': 0.265,
...`

This result seems sensible. For example, from the previous exercise one can easily see that 'AAU' and 'AAC' encode the same amino acid (N), and their relative frequencies sum to 1.

Now you should have everything in place to easily solve the following.


6. Write a class $\verb+ProteinToMaxRNA+$ with a $\verb+convert+$ method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with $\texttt{LTPIQNRA}$.

In [99]:
# exercise 6 DO NOT MODIFY THIS LINE
%pycat part07-e06_protein_to_max_rna/src/protein_to_max_rna.py

#!/usr/bin/env python3

CODON_PROBS = {'UUU': 0.46, 'UCU': 0.19, 'UAU': 0.44, 'UGU': 0.46, 'UUC': 0.54, 'UCC': 0.22, 'UAC': 0.56, 'UGC': 0.54, 'UUA': 0.08, 'UCA': 0.15, 'UAA': 0.29, 'UGA': 0.47, 'UUG': 0.13, 'UCG': 0.05, 'UAG': 0.24, 'UGG': 1.0, 'CUU': 0.13, 'CCU': 0.29, 'CAU': 0.42, 'CGU': 0.08, 'CUC': 0.2, 'CCC': 0.32, 'CAC': 0.58, 'CGC': 0.18, 'CUA': 0.07, 'CCA': 0.28, 'CAA': 0.26, 'CGA': 0.11, 'CUG': 0.4, 'CCG': 0.11, 'CAG': 0.74, 'CGG': 0.2, 'AUU': 0.36, 'ACU': 0.25, 'AAU': 0.47, 'AGU': 0.15, 'AUC': 0.47, 'ACC': 0.36, 'AAC': 0.53, 'AGC': 0.24, 'AUA': 0.17, 'ACA': 0.28, 'AAA': 0.43, 'AGA': 0.22, 'AUG': 1.0, 'ACG': 0.11, 'AAG': 0.57, 'AGG': 0.21, 'GUU': 0.18, 'GCU': 0.27, 'GAU': 0.46, 'GGU': 0.16, 'GUC': 0.24, 'GCC': 0.4, 'GAC': 0.54, 'GGC': 0.34, 'GUA': 0.12, 'GCA': 0.23, 'GAA': 0.42, 'GGA': 0.25, 'GUG': 0.46, 'GCG': 0.11, 'GAG': 0.58, 'GGG': 0.25}
AA_TO_RNA = {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA

### Idea of solution

In the convert method the input string is made into a list which is turned into an RNA sequence in a list comprehension. This done in the "Python private" __get_codon_with_max_prob__ helper method which gets an amino acid as input, produces a list of candidate codons, then joins it together with the appropriate probabilities to make a dict and returns the key with maximum value.

### Discussion
The output produced for 'LTPIQNRA' is 'CUGACCCCCAUCCAGAACAGAGCC'.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for $\texttt{AUU}$, $\texttt{AUC}$, and $\texttt{AUA}$, respectively. 
Let us assume we have a random number generator $\verb+random()+$ that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to [0,36/100), [36/100,83/100), [83/100,1), respectively. Depending which interval the number $\verb+random()+$ hits, we select the codon accordingly.

7. Write a function $\verb+random_event+$ that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function $\verb+random.uniform+$ to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [100]:
# exercise 7 DO NOT MODIFY THIS LINE
%pycat part07-e07_random_event/src/random_event.py

#!/usr/bin/env python3

import numpy as np


def random_event(dist):
    """Takes as input a dictionary from events to their probabilities.
Return a random event sampled according to the given distribution.
The probabilities must sum to 1.0"""
    # Write your solution here
    return np.random.choice(list(dist.keys()), 1, p=list(dist.values()))[0]

if __name__ == '__main__':
    distribution=dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(random_event(distribution))



### Idea of solution

Solution uses the ready-made numpy __choice__ method.

### Discussion

This could as easily have used the random.choice method that is part of the Python standard library, but I discovered the Numpy version first. 

With this general routine, the following should be easy to solve.
 
8. Write a class $\verb+ProteinToRandomRNA+$ to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the $\verb+convert+$ method. Run your program with $\texttt{LTPIQNRA}$.

In [101]:
# exercise 8 DO NOT MODIFY THIS LINE
%pycat part07-e08_protein_to_random_rna/src/protein_to_random_rna.py

#!/usr/bin/env python3

import numpy as np

CODON_PROBS = {'UUU': 0.46, 'UCU': 0.19, 'UAU': 0.44, 'UGU': 0.46, 'UUC': 0.54, 'UCC': 0.22, 'UAC': 0.56, 'UGC': 0.54, 'UUA': 0.08, 'UCA': 0.15, 'UAA': 0.29, 'UGA': 0.47, 'UUG': 0.13, 'UCG': 0.05, 'UAG': 0.24, 'UGG': 1.0, 'CUU': 0.13, 'CCU': 0.29, 'CAU': 0.42, 'CGU': 0.08, 'CUC': 0.2, 'CCC': 0.32, 'CAC': 0.58, 'CGC': 0.18, 'CUA': 0.07, 'CCA': 0.28, 'CAA': 0.26, 'CGA': 0.11, 'CUG': 0.4, 'CCG': 0.11, 'CAG': 0.74, 'CGG': 0.2, 'AUU': 0.36, 'ACU': 0.25, 'AAU': 0.47, 'AGU': 0.15, 'AUC': 0.47, 'ACC': 0.36, 'AAC': 0.53, 'AGC': 0.24, 'AUA': 0.17, 'ACA': 0.28, 'AAA': 0.43, 'AGA': 0.22, 'AUG': 1.0, 'ACG': 0.11, 'AAG': 0.57, 'AGG': 0.21, 'GUU': 0.18, 'GCU': 0.27, 'GAU': 0.46, 'GGU': 0.16, 'GUC': 0.24, 'GCC': 0.4, 'GAC': 0.54, 'GGC': 0.34, 'GUA': 0.12, 'GCA': 0.23, 'GAA': 0.42, 'GGA': 0.25, 'GUG': 0.46, 'GCG': 0.11, 'GAG': 0.58, 'GGG': 0.25}
AA_TO_RNA = {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU'

### Idea of solution

The implementation follows the previous exercise up the return value of helper method. In it, the returned value is a random amino acid instead of the one with the highest probability. The random amino acid is selected from the true probability distribution parsed from html file in previous exercises.

### Discussion
Verifying random/indeterministic output is difficult, but at least on the surface the function produces sensible results.

Example: output for protein_to_random_codons.convert("LTPIQNRA") is CUGACCCCUAUCCAGAACCGAGCA which and these codons do encode the respective amino acids.


## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* $\verb+sliding_window+$ to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function $\verb+sliding_window+$ to be a generator, it must have at least
  one $\verb+yield+$ expression, see https://docs.python.org/3/reference/expressions.html#yieldexpr.
  See also exercise 13 for an example of a generator $\verb+all_kmers+$ made with yield expressions.
  A yield expression can be used to return a value and *temporarily* return from the function.

In [102]:
# exercise 9 DO NOT MODIFY THIS LINE
%pycat part07-e09_sliding_window/src/sliding_window.py

#!/usr/bin/env python3

import sys
from collections import defaultdict

NUCLEOTIDES = 'ACGT'
def sliding_window(s, k):
    """This function return a generator that can be iterated over all
starting position of a k-window in the sequence.
For each starting position the generator returns the nucleotide frequencies
in the window as a dictionary."""
    idx = 0
    while(idx+k <= len(s)):
        kmer = s[idx:idx+k]
        letter_freqs = {str(nuc):kmer.count(nuc) for nuc in list(NUCLEOTIDES)}
        idx = idx + 1
        yield(letter_freqs)

if __name__ == '__main__':
    for d in sliding_window('GCGCTACGAT', 4):
        print(d)
    for s in sys.argv[1:]:
        for d in sliding_window(s, 4):
            print(d)



### Idea of solution

The generator walks along the input string and returns the letter frequencies of the k-length substring, where the substring moves towards the end by one at each iteration.

### Discussion
For, example, the output for sliding_window('GCGCTACGAT', 4) is produces 7 substrings which have the correct frequencies.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function $\verb+context_list+$ that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [103]:
# exercise 10 DO NOT MODIFY THIS LINE
%pycat part07-e10_context_list/src/context_list.py

#!/usr/bin/env python3

import sys
from collections import defaultdict

def kmer_gen(string, k):
    idx = 0
    while idx < len(string) - k:
        kmer = string[idx:idx+k]
        next_n = string[idx+k]
        idx += 1
        yield (kmer, next_n)

def context_list(s, k):
    contexts = defaultdict(str)
    for kmer, next_n in kmer_gen(s, k):
        contexts[kmer] = contexts[kmer] + next_n
    return dict(contexts)

if __name__ == '__main__':
    k=2
    if len(sys.argv) == 1:
        d=context_list("ATGATATCATCGACGATCTAG", k)
        print(d)
    else:
        for s in sys.argv[1:]:
            d=context_list(s, k)
            print(d)



### Idea of solution

The solution uses the generator kmer_gen which returns the current kmer and its next nucleotide as a tuple. Using defaultdict as the result initializes all values to an empty string. Using these two, the loop only has to concatenate next nucleotide given by the generator with the context of its preciding kmer, also given by the generator.

### Discussion
The output for context_list("ATGATATCATCGACGATCTAG", 2) is:

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}

11. With the above solution, write function $\verb+context_probabilities+$ to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k$ values 0 and 2.

In [104]:
# exercise 11 DO NOT MODIFY THIS LINE
%pycat part07-e11_context_probabilities/src/context_probabilities.py

#!/usr/bin/env python3

import sys
from collections import defaultdict

def kmer_gen(string, k):
    idx = 0
    while idx < len(string) - k:
        kmer = string[idx:idx+k]
        next_n = string[idx+k]
        idx += 1
        yield (kmer, next_n)

def context_list(s, k):
    kmers = defaultdict(str)
    for kmer, next_n in kmer_gen(s, k):
        kmers[kmer] = kmers[kmer] + next_n
    return dict(kmers)

NUCLEOTIDES = 'ATGC'

def context_probabilities(s, k):
    kmers = defaultdict(list)
    for kmer, context in context_list(s, k).items():
        kmers[kmer] = {str(n): context.count(n)/len(context) for n in list(NUCLEOTIDES)}
    return dict(kmers)

if __name__ == '__main__':
    k=2
    if len(sys.argv) > 1:
        for s in sys.argv[1:]:
            d=context_probabilities("ATGATATCATCGACGATGTAG", 2)
            print(d)
    else:
        s = "ATGATATCATCGACGATGTAG"
        d = context_probabilities(s, k)
        from pprint import PrettyPrinter
        PrettyPrinter().pprint(d

### Idea of solution

The **context_probabilities** function uses the context_list generator which is casts to a list. This is then looped through produce the probabilities for each nucleotide using a dict comprehension. In it, each nucleotide in 'ATCG' becomes the key and the value is the number of occurrences of that nucleotide divided by length of the context.

### Discussion
The output for context_probabilities("ATGATATCATCGACGATGTAG", 2) is: 

`{'AT': {'A': 0.2, 'T': 0.0, 'G': 0.4, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5, 'G': 0.0, 'C': 0.0}, 'GA': {'A': 0.0, 'T': 0.6666666666666666, 'G': 0.0, 'C': 0.3333333333333333}, 'TA': {'A': 0.0, 'T': 0.5, 'G': 0.5, 'C': 0.0}, ..}`


This looks ok, because the probabilities of nucleotides for each kmer sum to one. Also, by looking at, say, kmer 'TA', which appears in the input string twice, the probabilities for T are 0.5 which is correct because these are indeed the succeeding nucleotides for the 'TA' in the string.

12. With the above solution and the function $\verb+random_event+$ from the earlier exercise, write class $\verb+MarkovChain+$. Its $\verb+generate+$ method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [105]:
# exercise 12 DO NOT MODIFY THIS LINE
%pycat part07-e12_generate_markov/src/generate_markov.py

#!/usr/bin/env python3

import sys
import numpy as np
#from collections import OrderedDict

k=2

zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}

kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
       'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
       'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
       'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
       'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
       'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
       'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
       'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
       'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}


def random_event(dist):
    """Takes as input a dictionary from events to their probabilities.
Return a random event sampled according to the given distribution.
The probabilities must sum to 1.0"""
    probs = np.array(list(dist.values()))
    #normalize because due to rounding,
    #probabilities do not sum to exactly 1

### Idea of solution

The Markov chain generation function first checks if there required chain lenght (n) is less than the kmer, and if it is, returns a string consisting of individually drafted nucleotides using the zeroth dict. 

Otherwise, it first drafts a kmer as a starting point and then adds nucleotides to it one by one using the kth dict probabilities. The latter step is done n - k times to account for the fact that it started with a kmer, making the output string length n. 

### Discussion
Example output for mc.generate(10, 0) is 'TGTATGTATC'. This seems correctly because according to the probabilitites in kth dict the kmer 'GT' can only be succeeded by A, and this is true for the output string. 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Modify the previous solution 11 to use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. 
You may use the provided generator $\verb+all_kmers+$ to iterate over all $k$-mer of given length.

In [106]:
# exercise 13 DO NOT MODIFY THIS LINE
%pycat part07-e13_pseudocounts/src/pseudocounts.py

#!/usr/bin/env python3

import sys
from collections import defaultdict
import numpy as np
from pprint import PrettyPrinter

NUCLEOTIDES = ['A', 'T', 'G', 'C']
#assign prob 0.25 to each nucleotide when preceding kmer not found:
MISSING_KMER_PROB = dict(zip(NUCLEOTIDES, [1/len(NUCLEOTIDES)]*len(NUCLEOTIDES)))

def all_kmers(k):
    """A generator that can be used to iterate through all k-mers.
Usage: for kmer in all_kmers(k):
           do_something(kmer)
"""
    if k==0:
        yield ""
    else:
        nucs= "".join(NUCLEOTIDES)
        v=[0]*k
        v[-1]=-1
        limit=pow(4, k)
        for i in range(limit):
            i=k-1
            while v[i] == 3:
                v[i] = 0
                i -= 1
            v[i] += 1
            yield "".join([ nucs[x] for x in v ])

def kmers_in_str(string, k):
    idx = 0
    while idx < len(string) - k:
        kmer = string[idx:idx+k]
        next_n = string[idx+k]
        idx += 1
        yield (kmer, next_n)

def context_list(s, k)

### Idea of solution
First all contexts are computed using the **context_list** function. Thes the **all_kmers** generator is used to loop through all the possible kmers. If the kmer is found in the contexts, the pseudo probabilities are computed using **et_pseudo_probs** function. If not, the MISSING_KMER_PROB is assigned for the kmer. This gives 0.25 probability for each nucleotide, which is also the pseudo probability for uniform probability.


### Discussion

Part of output for context_probabilities("ATGATATCATCGACGATGTAG", 2) is:

`{'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AT': {'A': 0.222, 'C': 0.333, 'G': 0.333, 'T': 0.111},
 'CA': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4},
 ..`
 
 By looking at it, one can observe some correct properties, namely:
 * The nucleotide probabilities for each kmer sum to 1 (give or take some rounding).
 * Nucleotides for kmers which are not present in the input string have have 0.25 probability.

14. Write class $\verb+MarkovChain+$ that given the $k$-th order Markov chain developed above to the constructor, its method $\verb+probability+$ computes the probability of a given input DNA sequence.

In [107]:
# exercise 14 DO NOT MODIFY THIS LINE
%pycat part07-e14_markov_chain_probability/src/markov_chain_probability.py

#!/usr/bin/env python3

import sys
import numpy as np

kth = {'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AT': {'A': 0.2222222222222222,
        'C': 0.3333333333333333,
        'G': 0.3333333333333333,
        'T': 0.1111111111111111},
 'CA': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4},
 'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CG': {'A': 0.5,
        'C': 0.16666666666666666,
        'G': 0.16666666666666666,
        'T': 0.16666666666666666},
 'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GA': {'A': 0.14285714285714285,
        'C': 0.2857142857142857,
        'G': 0.14285714285714285,
        'T': 0.42857142857142855},
 'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GT': {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2},
 'TA': {'A': 0.16666666666666666,
        'C': 0.16666666666666666,
        'G': 0.3

### Idea of solution

First the probability of the initial kmer is computed by multiplying the probabilities of the individual nucleotides, as we don't have preceding kmer information from which we could produce better estimates. Them **__kmers_with_next_nucs** generator handles the rest where we only have to multiply the initial probability with the probability of the succeeding nucleotide in that context, using the kth dict, and continue similarly through the whole input string.

### Discussion
According to the function, the probability of 'ATGATATCATCGACGATGTAG' is 2.831270e-10.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class $\verb+MarkovChain+$ that given the $k$-th order Markov chain developed above to the constructor, its method $\verb+log_probability+$ computes the log-probability of a given input DNA sequence. Run your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k=2$.

In [108]:
# exercise 15 DO NOT MODIFY THIS LINE
%pycat part07-e15_markov_chain_log_probability/src/markov_chain_log_probability.py

#!/usr/bin/env python3

import sys
from math import log2

kth = {'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AT': {'A': 0.2222222222222222,
        'C': 0.3333333333333333,
        'G': 0.3333333333333333,
        'T': 0.1111111111111111},
 'CA': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4},
 'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CG': {'A': 0.5,
        'C': 0.16666666666666666,
        'G': 0.16666666666666666,
        'T': 0.16666666666666666},
 'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GA': {'A': 0.14285714285714285,
        'C': 0.2857142857142857,
        'G': 0.14285714285714285,
        'T': 0.42857142857142855},
 'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GT': {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2},
 'TA': {'A': 0.16666666666666666,
        'C': 0.16666666666666666,
        'G': 

### Idea of solution

The solution is essentially the same as the previous one with two distinctions: 
* In the constructor, all probabilities are transformed to log 2 probabilities.
* When calculating the output probability, sum is used instead of multiplication.

### Discussion
According to the function, the log probability of 'ATGATATCATCGACGATGTAG' is -31.71783. When it used as exponent for 2, is 2.8312731645657305e-10 which is result of the previous exercise, as it should.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  $\verb+context_probabilities+$ function.

In [109]:
# exercise 16 DO NOT MODIFY THIS LINE
%pycat part07-e16_low_space_requirement/src/low_space_requirement.py

#!/usr/bin/env python3

import sys
from collections import defaultdict
import regex as re #use regex (not re) to support overlapping matches TA

NUCLEOTIDES = 'ATGC'

#assign prob 0.25 to each nucleotide. this is used when preceding kmer is not found.
MISSING_KMER_PROB = dict(zip(NUCLEOTIDES, [1/len(NUCLEOTIDES)]*len(NUCLEOTIDES)))

def all_kmers(k):
    v=[0]*k
    v[-1]=-1
    limit=pow(4, k)
    for i in range(limit):
        i=k-1
        while v[i] == 3:
            v[i] = 0
            i -= 1
        v[i] += 1
        yield "".join([ NUCLEOTIDES[x] for x in v ])

def get_pseudo_probs(context):
    """
    returns a dict with nucleotides as keys. The value is the
    pseudo probability of that nucleotide where 1 is added to the
    frequency. Nucleotides which are not found get a frequency of 1.
    """
    pseudo_total = len(context) + len(NUCLEOTIDES)
    return {str(nuc): (context.count(nuc) + 1)/pseudo_total for nuc in list(NUCLEOTIDES)}

def context_probabilities(s, k):
    k

### Idea of solution

In the solution all kmers are iterated through with the **all_kmers** generator. If the kmer is found in the input string, its context produced by using a regex that incorporates the kmer inside it and matches the next letter after it. The solution uses a Python library called 'regex' instead of the usual 're' to support overlapping matches (e.g kmer "AT" in input string "ATAT"). After the context is found, the **get_pseudo_probs** method is used with essentially the same functionality as in previous exercises.

### Discussion
Part of the output for context_probabilities("ATGATATCATCGACGATGTAG")
`'AA': {'A': 0.25, 'T': 0.25, 'G': 0.25, 'C': 0.25}, 'AT': {'A': 0.2222222222222222, 'T': 0.1111111111111111, 'G': 0.3333333333333333, 'C': 0.3333333333333333}, 'AG': {'A': 0.25, 'T': 0.25, 'G': 0.25, 'C': 0.25}, 'AC': {'A': 0.2, 'T': 0.2, 'G': 0.4, 'C': 0.2} 
..`

This looks to be the same as the output of the pseudo probabilities exercise, so all good there!


While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. You may use the function $\verb+random.choice+$.

In [110]:
# exercise 17 DO NOT MODIFY THIS LINE
%pycat part07-e17_sample_from_concatenation/src/sample_from_concatenation.py

#!/usr/bin/env python3

import sys
import random
from collections import defaultdict
import numpy as np
import regex as re #use regex (not re) to support overlapping matches TA

NUCLEOTIDES = "ACGT"
#assign prob 0.25 to each nucleotide. this is used when preceding kmer is not found.
MISSING_KMER_PROB = dict(zip(NUCLEOTIDES, [1/len(NUCLEOTIDES)]*len(NUCLEOTIDES)))

def all_kmers(k):
    """A generator that can be used to iterate through all k-mers.
Usage: for kmer in all_kmers(k):
           do_something(kmer)
"""
    if k==0:
        yield ""
    else:
        v=[0]*k
        v[-1]=-1
        limit=pow(4, k)
        for i in range(limit):
            i=k-1
            while v[i] == 3:
                v[i] = 0
                i -= 1
            v[i] += 1
            yield "".join([ NUCLEOTIDES[x] for x in v ])

class MarkovChain(object):

    def __init__(self, s, k):
        self.k = k
        self.s = s
        self.contexts = self.__get_contexts(s, k)

    def __get_contexts(self, s,

### Idea of solution

The constructor produces the contexts for all kmers in the input string using the solution from exercise 16. The generate method uses the **all_kmers** iterator to loop throught all possible kmers, gets the context for each and samples a nucleotide from it using the **random.choice** method. The starter kmer for the Markov chain is sampled from all kmers found in the input string (which are the keys of the contexts dict).

### Discussion
An example output for input string "ATGATATCATCGACGATGTAG", k=3 and n=10 is 

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function $\verb+kmer_index+$ by modifying your earlier code for the $k$-th order Markov chain. Test your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k=2$.

In [111]:
# exercise 18 DO NOT MODIFY THIS LINE
%pycat part07-e18_kmer_index/src/kmer_index.py

#!/usr/bin/env python3

import sys
from collections import defaultdict


def sliding_window(s, k):
    """This function returns a generator that can be iterated over all
starting position of a k-window in the sequence."""
    idx = 0
    while(idx+k <= len(s)):
        kmer = s[idx:idx+k]
        yield(kmer, idx)
        idx = idx + 1

def kmer_index(s, k):
    indices = defaultdict(list)
    for kmer, idx in sliding_window(s, k):
        indices[kmer].append(idx)
    return indices

if __name__ == '__main__':
    k=2
    try:
        s = sys.argv[1]
    except IndexError:
        s = "ATGATATCATCGACGATGTAG"
    try:
        k = int(sys.argv[2])
    except (IndexError, ValueError):
        k = 2
    print("Using string:")
    print(s)
    print("".join(["%i" % (i%10) for i in range(len(s))]))
    print("%i-mer index is:" % k)
    d=kmer_index(s, k)
    print(dict(d))



### Idea of solution

The solution uses the **sliding_window** generator from a previous exercise, but instea of letter frequencies, the kmer and its index is return as a tuple. This makes it easy to produce the output dict as the loop only has to append the index to the correct existing index list for that kmer. 

### Discussion
Output for kmer_index("ATGATATCATCGACGATGTAG") is:
`{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}`

Seems correct!

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P|Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function $\verb+kullback_leibler+$ to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.

In [112]:
# exercise 19 DO NOT MODIFY THIS LINE
%pycat part07-e19_kullback_leibler/src/kullback_leibler.py

#!/usr/bin/env python3

from collections import defaultdict
from math import log2
import random
aas="*ACDEFGHIKLMNPQRSTVWY"

aa_to_codons = {'*': ['UAA', 'UGA', 'UAG'],
 'A': ['GCU', 'GCC', 'GCA', 'GCG'],
 'C': ['UGU', 'UGC'],
 'D': ['GAU', 'GAC'],
 'E': ['GAA', 'GAG'],
 'F': ['UUU', 'UUC'],
 'G': ['GGU', 'GGC', 'GGA', 'GGG'],
 'H': ['CAU', 'CAC'],
 'I': ['AUU', 'AUC', 'AUA'],
 'K': ['AAA', 'AAG'],
 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'],
 'M': ['AUG'],
 'N': ['AAU', 'AAC'],
 'P': ['CCU', 'CCC', 'CCA', 'CCG'],
 'Q': ['CAA', 'CAG'],
 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'],
 'T': ['ACU', 'ACC', 'ACA', 'ACG'],
 'V': ['GUU', 'GUC', 'GUA', 'GUG'],
 'W': ['UGG'],
 'Y': ['UAU', 'UAC']}
codon_to_aa = {'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', '

### Idea of solution

I was not sure what this exercise had in mind for the test data generation, so I ended up implementing just the **kullback_leibler** function. I was also confused with the tests which seemed to both require the function the raise an exception and return value 2.0 for inputs
`
p = dict(zip("ACGT", [1.0, 0.0, 0.0, 0.0]))
q = dict(zip("ACGT", [0.25]*4))
`
I couldn't satisfy both conditions, so I opted to return return a value and catch the exception.


### Discussion
fill in